In [ ]:
# Connecting to Azure ML Workspace
from azureml.core import Workspace, Model

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, sep=' | ')

# Loading the model from the Azure ML registry
model = Model(workspace=ws, name="pimadiabetes_logistic_regression_model")
print("Model loaded:", model.name)



In [ ]:
# Definition of the inference environment
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.from_pip_requirements(name="pima-env", file_path="requirements.txt")

inference_config = InferenceConfig(
    entry_script="score.py",
    environment=env
)


In [ ]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(
    workspace=ws,
    name="pima-diabetes-deploy-04",  
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config
)

service.wait_for_deployment(show_output=True)


In [ ]:
# Testing the API with a prediction request
import requests
import json

sample_data = json.dumps({"data": [[8, 183, 64, 29.15, 155.54, 23.3, 0.672, 32]]})
headers = {"Content-Type": "application/json"}

response = requests.post(service.scoring_uri, data=sample_data, headers=headers)
result = response.json()
print(result["message"])
print(result["explanation"])
